In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [133]:
# ELECTRICITY GENERATION DATA
elec_gen_raw = pd.read_csv('data/elec_gen_monthly_cleaned.csv')
elec_gen_raw = elec_gen_raw[['Year', 'Electricity Supply: Total: Gross']].copy()
elec_gen_raw['date'] = pd.to_datetime(elec_gen_raw['Year'], format='%m/%Y')
elec_gen_raw = elec_gen_raw.rename(columns={'Electricity Supply: Total: Gross': 'total_gen'})
elec_gen_raw['tot_moving_avg'] = elec_gen_raw['total_gen'].rolling(window=30, min_periods=1).mean()

gen2003 = elec_gen_raw[elec_gen_raw['date'].dt.year >= 2003]
gen2003 = gen2003.drop(['Year'], axis=1)
gen2003.set_index('date', inplace=True)


In [135]:
# INSTALLED CAPACITY DATA

ic_raw = pd.read_csv("data/installed_capacity_annual_2023.csv")
ic = ic_raw.loc[:20, ['year', 'grand_total']].copy()
ic['year'] = pd.to_datetime(ic['year'], format='%Y')
ic = ic.rename(columns={'grand_total':'total_ic'})
ic.set_index('year', inplace=True)

# Make Monthly
ic_m = ic.resample('MS').ffill()

# Add last row to complete 2023
row = ic.iloc[-1]
date_range = pd.date_range(start='2023-02-01', end='2024-05-31', freq='MS')

for date in date_range:
    ic_m.loc[date] = row.copy()

ic_m['total_ic'] = pd.to_numeric(ic_m.loc[:, 'total_ic'].str.strip('').str.replace(',', ''))

In [139]:
# MAKE COMBINED

gc = gen2003.join(ic_m)
gc['gen_cap_ratio'] = gc['total_gen'] / gc['total_ic']
gc['moving_avg_cap_ratio'] = gc['tot_moving_avg'] / gc['total_ic']

In [140]:
gc

,total_gen,tot_moving_avg,total_ic,gen_cap_ratio,moving_avg_cap_ratio
date,,,,,
2003-01-01,15464.98,16092.219333,40086.0,0.385795,0.401442
2003-02-01,13838.03,15968.357667,40086.0,0.345209,0.398352
2003-03-01,16008.95,15960.447667,40086.0,0.399365,0.398155
2003-04-01,16511.63,15954.803000,40086.0,0.411905,0.398014
2003-05-01,17780.61,16052.672667,40086.0,0.443562,0.400456
...,...,...,...,...,...
2024-01-01,21888.41,23889.544667,64044.6,0.341768,0.373014
2024-02-01,19649.40,23649.330000,64044.6,0.306808,0.369263
2024-03-01,22498.68,23518.441667,64044.6,0.351297,0.367220


In [141]:
gc.to_csv("gen_cap_merged.csv")